In [1]:
'''
import library 
'''
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!pip install -U opencv-contrib-python==3.4.2.17
import cv2

In [16]:
'''
Function (modify)
'''
from collections import namedtuple
Pt = namedtuple('Point', 'x y')

def compute_homography(p1, p2):	
    """
    compute the homography transform
    p1, p2: 4 x 2, 4 groups of corresponding points
    """	
    # IMPLEMENT HERE, hint: you may find np.linalg.svd useful
    p1, p2 = [Pt(x[0], x[1]) for x in p1], [Pt(x[0], x[1]) for x in p2]



    # construct the A matrix in the same way in the slides
    A = []
    for i in range(len(p1)):
        A.append([p2[i].x, p2[i].y, 1, 0, 0, 0, -p2[i].x*p1[i].x, -p2[i].y*p1[i].x, -p1[i].x])
        A.append([0, 0, 0, p2[i].x, p2[i].y, 1, -p2[i].x*p1[i].y, -p2[i].y*p1[i].y, -p1[i].y])
    A = np.matrix(A)

    
    # use SVD to find the solution to Ax = 0
    u, s, vh = np.linalg.svd(A)
    v = vh.T
    h = v[:,-1]/v[-1,-1]
    H = h.reshape(3,3)
    

    # reorgnize the solution vector and normalize it to get homography matrix
    return H

In [15]:
## Test ##
p1 = np.array([[366.6972,  106.9789],
  [439.9366,   84.4437],
  [374.5845,  331.2042],
  [428.6690,  326.6972]])

p2 = np.array([ [115.0000,  120.0000],
  [194.0000,  107.0000],
  [109.0000,  351.0000],
  [169.0000,  346.0000] ])
H = compute_homography(p1, p2)

assert np.allclose(H, np.array([[ 6.76097539e-01,  9.12020085e-02,  2.61445474e+02],
       [-1.57581282e-01,  9.59039559e-01,  5.18593524e+00],
       [-4.93549913e-04,  9.67378471e-05,  1.00000000e+00]]))

(8, 9)
[[ 6.76097539e-01  9.12020085e-02  2.61445474e+02]
 [-1.57581282e-01  9.59039559e-01  5.18593524e+00]
 [-4.93549913e-04  9.67378471e-05  1.00000000e+00]]


In [ ]:
def plot_corr(I1, I2, p1, p2):
  """
  utility function to plot correspondence between p1 in I1 and p2 in I2

  I1, I2: H * W, two input images of the same size
  p1, p2: N * 2, points on two images 
  """
  I = np.hstack((I1, I2))
  sy,sx = I1.shape[0:2]

  plt.figure()
  plt.imshow(I)
  plt.plot(p1[:, 0], p1[:, 1],'bo')
  plt.plot(sx + p2[:, 0], p2[:, 1],'rx')
  plt.plot(np.c_[p1[:, 0], sx + p2[:, 0]].T, np.c_[p1[:, 1],p2[:, 1]].T, 'g-')
  plt.show()

In [58]:
'''
Function (modify)
'''
def match_features(f1,f2):
    """
    f1, f2: N * feature_size, two features to be matched
    """

    # IMPLEMENT HERE
    # hint: you may find cdist from scipy.spatial.distance, numpy.argsort useful

    # compute pairwise distance between f1 and f2
    dist_fwd = cdist(f1, f2)
    dist_back = dist_fwd.T

    # forward matching, find closet two matches in f2 to f1 and do a ratio test
    print(dist_fwd)
    potential_matches_fwd = list(zip(*np.unravel_index(np.argsort(dist_fwd, axis=None), dist_fwd.shape)))
    
    matches_fwd = [x for x in potential_matches_fwd if dist_fwd[potential_matches_fwd[0]] / dist_fwd[x] < 0.7]
    matches_fwd = [dist_fwd[potential_matches_fwd[0]] / dist_fwd[x] for x in potential_matches_fwd]
    print(matches_fwd)
    
#     dist_fwd[match1] = float('inf')
#     match2 = np.unravel_index(np.argmin(dist_fwd, axis=None), dist_fwd.shape)
    print(match1)
    matches_back = list(zip(*np.unravel_index(np.argsort(dist_back, axis=None), dist_back.shape)))
    assert dist_fwd[matches_back[0]] / dist_fwd[matches_back[1]] < 0.7

    # backward matching, find closet two matches in f1 to f2 and do a ratio test

    # get the intersect of two matching as the final result, python set 

    return match, match_fwd, match_bkwd

In [59]:
## Test ##
a = np.array([[2, 3],[5, 8],[4, 10]])
b = np.array([[10, 2],[2, 3],[5, 7],[1, -10]])
match, match_fwd, match_bkwd = match_features(a, b)
assert np.allclose(match, np.array([[0, 1],[1, 2]]))
assert np.allclose(match_fwd, np.array([[0, 1],[1, 2],[2, 2]]))
assert np.allclose(match_bkwd, np.array([[0, 1],[1, 2]]))

[[ 8.06225775  0.          5.         13.03840481]
 [ 7.81024968  5.83095189  1.         18.43908891]
 [10.          7.28010989  3.16227766 20.22374842]]
[nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


NameError: name 'match1' is not defined

In [17]:
"""
Script (do not modify)
"""

# load images in OpenCV BGR format
I1 = cv2.imread('yosemite1.jpg')
I2 = cv2.imread('yosemite2.jpg')

# create grayscale images
I1_gray = cv2.cvtColor(I1, cv2.COLOR_BGR2GRAY)
I2_gray = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)

# convert images to RGB format for display
I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2RGB)
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2RGB)

# compute SIFT features
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(I1_gray, None)
kp2, des2 = sift.detectAndCompute(I2_gray, None)

# match features
match, match_fwd, match_bkwd = match_features(des1, des2)

# get corresponding points p1, p2 
p1 = np.array([kp.pt for kp in kp1])[match[:, 0]]
p2 = np.array([kp.pt for kp in kp2])[match[:, 1]]

# plot first 20 matching points 
plot_corr(I1, I2, p1[:20], p2[:20])

NameError: name 'match_features' is not defined

In [ ]:
'''
Function (modify)
'''
def ransac_homography(p1, p2):
  """
  p1, p2: N * 2, correponding point coordinates
  """
  
  # IMPLEMENT HERE
  
  # decide how many loops to run and what the threshold is

  # RANSAC loop, inside in the loop
  #     1. randomly pick n samples from p1 and p2 that is enough to fit a model(n=4 here)

  #     2. fit the model and get a estimation

  #     3. count consensus(# of inliners that is close to the estimatiion within the threshold)

  #     4. update the best estimation if the current one is better 
  return best_H

In [ ]:
## Test ##
p2 = np.random.randint(20, 200, size=(10, 2))
p2_ = np.hstack((p2, np.ones((10, 1)))).T
H = np.array([[ 6.76097539e-01,  9.12020085e-02,  2.61445474e+02],
       [-1.57581282e-01,  9.59039559e-01,  5.18593524e+00],
       [-4.93549913e-04,  9.67378471e-05,  1.00000000e+00]])
p1_ = H @ p2_
p1 = p1_ / p1_[-1, :]
p1 = p1[:2]
p1[:, -1] += 50
p1 = p1.T
assert np.allclose(H, ransac_homography(p1, p2))

In [ ]:
"""
Function (do not modify)
"""
def stitch(I1, I2, H):
    """
    utility function to stitch two images

    Input:

    I1: H1 x W1 x 3, target image
    I2: H2 x W2 x 3, source image
    H: homography matrix such that I1 = H @ I2

    Output:
    I_: stitching result
    """
    
    # map I2 to I1
    
    # create new dimensions to accomodate points from I2

    # create images I_ for mapping

    # map I_ to I2 (translation then homography)

    # keep only the valid coordinates of I2

    # in blend area, take the average of I1_ and I2_
    
    return I_

In [ ]:
"""
Script (do not modify)
"""

# load images in OpenCV BGR format
I1 = cv2.imread('yosemite1.jpg')
I2 = cv2.imread('yosemite2.jpg')

# create grayscale images
I1_gray = cv2.cvtColor(I1, cv2.COLOR_BGR2GRAY)
I2_gray = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)

# convert images to RGB format for display
I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2RGB)
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2RGB)

# compute SIFT features
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(I1_gray, None)
kp2, des2 = sift.detectAndCompute(I2_gray, None)

# match features
match, match_fwd, match_bkwd = match_features(des1, des2)

# get corresponding points p1, p2 
p1 = np.array([kp.pt for kp in kp1])[match[:, 0]]
p2 = np.array([kp.pt for kp in kp2])[match[:, 1]]

# plot first 20 matching points 
plot_corr(I1, I2, p1[:20], p2[:20])

# estimate homography transform with RANSAC
H = ransac_homography(p1, p2)

# stitch two images together and show the results
I = stitch(I1,I2,H)
plt.figure()
plt.imshow(I)
plt.show()